# 1C Customers dynamic features


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
import pandas as pd

sys.path.append("..")

from src.utils.core_utils import setup_logging

# Initialize logging
file_log = "customer_dynamic_feature.log"
root_logger = setup_logging(level=logging.DEBUG, log_file=file_log, remove_existing=True)

2025-06-05 12:21:28,401 - src.utils.core_utils - INFO - Creating file handler with level: 10
2025-06-05 12:21:28,402 - src.utils.core_utils - DEBUG - Logging setup complete to customer_dynamic_feature.log


In [3]:
from src.feature_customers import CustomerDynamicFeaturePipelineConfig, CustomerDynamicFeaturePipeline

/Users/sharonshiao/Dropbox/Machine_learning/projects/fahsion_recommendation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run pipeline


In [4]:
customer_dynamic_feature_config = CustomerDynamicFeaturePipelineConfig.create_default()

In [5]:
customer_dynamic_feature_pipeline = CustomerDynamicFeaturePipeline(customer_dynamic_feature_config)

In [6]:
customer_dynamic_feature_pipeline.setup()

2025-06-05 12:21:35,183 - src.feature_customers - INFO - Setting up CustomerDynamicFeaturePipeline
2025-06-05 12:21:35,184 - src.feature_customers - DEBUG - Config: {
  "config_processor": {
    "history_start_week_num": 52,
    "history_end_week_num": 104,
    "start_week_num": 92,
    "end_week_num": 104,
    "k_items": 5
  },
  "subsample": 0.25,
  "seed": 42
}


In [7]:
results_customer_dynamic_feature = customer_dynamic_feature_pipeline.run()

2025-06-05 12:21:35,205 - src.feature_customers - INFO - Loading data for CustomerDynamicFeaturePipeline
2025-06-05 12:21:35,205 - src.feature_extraction - INFO - Loading optimized raw data from transactions train 0.25 42
2025-06-05 12:21:35,206 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_train_sample_0.25_42.parquet
2025-06-05 12:21:35,471 - src.feature_extraction - INFO - Loading optimized raw data from transactions valid 0.25 42
2025-06-05 12:21:35,472 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_valid_sample_0.25_42.parquet
2025-06-05 12:21:35,474 - src.feature_extraction - INFO - Loading optimized raw data from transactions test 0.25 42
2025-06-05 12:21:35,475 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_test_sample_0.25_42.parquet
2025-06-05 12:21:35,546 - src.features_articles - INFO - Loading article embedding result from ../data/preprocessed/articles_embedding/full
2025-06-05 12:21:35,637 -

# Checks


In [8]:
results_customer_dynamic_feature.data.info()
print(results_customer_dynamic_feature.data.shape)
display(results_customer_dynamic_feature.data.head())
print(results_customer_dynamic_feature.feature_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710527 entries, 0 to 1710526
Data columns (total 5 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   customer_id                   uint64 
 1   week_num                      int64  
 2   customer_avg_price            float64
 3   customer_avg_text_embedding   object 
 4   customer_avg_image_embedding  object 
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 65.3+ MB
(1710527, 5)


,customer_id,week_num,customer_avg_price,customer_avg_text_embedding,customer_avg_image_embedding
0,41318098387474,92,0.022017,"[0.005770983, -0.42972183, 0.15940754, 0.12657...","[1.4321077, 0.20973971, 0.09196462, 1.108316, ..."
1,41318098387474,93,0.022017,"[0.005770983, -0.42972183, 0.15940754, 0.12657...","[1.4321077, 0.20973971, 0.09196462, 1.108316, ..."
2,41318098387474,94,0.022017,"[0.005770983, -0.42972183, 0.15940754, 0.12657...","[1.4321077, 0.20973971, 0.09196462, 1.108316, ..."
3,41318098387474,95,0.022017,"[0.005770983, -0.42972183, 0.15940754, 0.12657...","[1.4321077, 0.20973971, 0.09196462, 1.108316, ..."
4,41318098387474,96,0.022017,"[0.005770983, -0.42972183, 0.15940754, 0.12657...","[1.4321077, 0.20973971, 0.09196462, 1.108316, ..."


{'numerical_features': ['customer_avg_price'], 'embedding_features': ['customer_avg_text_embedding', 'customer_avg_image_embedding'], 'one_hot_features': [], 'categorical_features': [], 'id_columns': ['customer_id', 'week_num']}


In [9]:
# Check the shape of the dataframe should match number of customers * number of weeks
print(results_customer_dynamic_feature.data.shape)
print(results_customer_dynamic_feature.data.customer_id.nunique())
print(results_customer_dynamic_feature.data.week_num.value_counts())

(1710527, 5)
131579
week_num
92     131579
93     131579
94     131579
95     131579
96     131579
97     131579
98     131579
99     131579
100    131579
101    131579
102    131579
103    131579
104    131579
Name: count, dtype: int64


## Check that the avg embeddings make sense


In [10]:
import numpy as np

from src.feature_customers import CustomerDynamicFeatureProcessor
from src.feature_extraction import load_optimized_raw_data
from src.features_articles import ArticleEmbeddingResult, get_path_to_article_features
from src.utils.data_checks import test_feature_customer_avg_embedding, test_feature_customer_avg_embedding_pipeline

2025-06-05 12:25:33,305 - matplotlib - DEBUG - matplotlib data path: /Users/sharonshiao/Dropbox/Machine_learning/projects/fahsion_recommendation/venv/lib/python3.12/site-packages/matplotlib/mpl-data
2025-06-05 12:25:33,316 - matplotlib - DEBUG - CONFIGDIR=/Users/sharonshiao/.matplotlib
2025-06-05 12:25:33,529 - matplotlib - DEBUG - interactive is False
2025-06-05 12:25:33,530 - matplotlib - DEBUG - platform is darwin
2025-06-05 12:25:33,672 - matplotlib - DEBUG - CACHEDIR=/Users/sharonshiao/.matplotlib
2025-06-05 12:25:33,680 - matplotlib.font_manager - DEBUG - Using fontManager instance from /Users/sharonshiao/.matplotlib/fontlist-v390.json


In [11]:
subsample = customer_dynamic_feature_config.subsample
seed = customer_dynamic_feature_config.seed
transactions_train = load_optimized_raw_data(data_type="transactions", sample="train", subsample=subsample, seed=seed)
transactions_valid = load_optimized_raw_data(data_type="transactions", sample="valid", subsample=subsample, seed=seed)
transactions_test = load_optimized_raw_data(data_type="transactions", sample="test", subsample=subsample, seed=seed)
path_article_embeddings = get_path_to_article_features(feature_type="embedding", subsample=1, seed=seed)
results_article_embeddings = ArticleEmbeddingResult.load(path_to_dir=path_article_embeddings)

2025-06-05 12:25:35,075 - src.feature_extraction - INFO - Loading optimized raw data from transactions train 0.25 42
2025-06-05 12:25:35,076 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_train_sample_0.25_42.parquet
2025-06-05 12:25:35,298 - src.feature_extraction - INFO - Loading optimized raw data from transactions valid 0.25 42
2025-06-05 12:25:35,298 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_valid_sample_0.25_42.parquet
2025-06-05 12:25:35,315 - src.feature_extraction - INFO - Loading optimized raw data from transactions test 0.25 42
2025-06-05 12:25:35,316 - src.feature_extraction - DEBUG - Loading data from ../data/transactions_test_sample_0.25_42.parquet
2025-06-05 12:25:35,326 - src.features_articles - INFO - Loading article embedding result from ../data/preprocessed/articles_embedding/full
2025-06-05 12:25:35,630 - src.features_articles - INFO - Loaded text embeddings from ../data/preprocessed/articles_embedding/full/

In [12]:
transactions = pd.concat([transactions_train, transactions_valid, transactions_test], ignore_index=True)
del transactions_train, transactions_valid, transactions_test

In [13]:
# Test metadata of pipeline
test_feature_customer_avg_embedding_pipeline(
    results_customer_dynamic_feature,
    transactions,
    customer_dynamic_feature_config,
)

Checks passed


In [14]:
# Test accuracy of avg embeddings
for customer_id in results_customer_dynamic_feature.data.customer_id.unique()[:5]:
    test_feature_customer_avg_embedding(
        results_customer_dynamic_feature,
        transactions,
        results_article_embeddings,
        customer_dynamic_feature_config,
        customer_id,
    )
    print("=" * 80)

Customer 41318098387474 has 13 weeks: [ 92  93  94  95  96  97  98  99 100 101 102 103 104]


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
6822314,2020-06-17,41318098387474,688728023,0.025407,1,91
6822315,2020-06-17,41318098387474,842605012,0.025407,1,91
6822316,2020-06-17,41318098387474,858918004,0.025407,1,91
6552190,2020-05-25,41318098387474,579541075,0.016932,1,87
6552191,2020-05-25,41318098387474,768931001,0.016932,1,87


Calculating avg price and embeddings for customer 41318098387474 and week 92
Checks for customer 41318098387474 and week 92 passed


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
6822314,2020-06-17,41318098387474,688728023,0.025407,1,91
6822315,2020-06-17,41318098387474,842605012,0.025407,1,91
6822316,2020-06-17,41318098387474,858918004,0.025407,1,91
6552190,2020-05-25,41318098387474,579541075,0.016932,1,87
6552191,2020-05-25,41318098387474,768931001,0.016932,1,87


Calculating avg price and embeddings for customer 41318098387474 and week 93
Checks for customer 41318098387474 and week 93 passed


...

Calculating avg price and embeddings for customer 857913002275398 and week 102
Checks for customer 857913002275398 and week 102 passed


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
7580313,2020-08-15,857913002275398,636207006,0.033881,1,99
7580314,2020-08-15,857913002275398,863456003,0.025407,1,99
7498043,2020-08-07,857913002275398,720504001,0.033881,1,98
7498044,2020-08-07,857913002275398,720504008,0.033881,1,98
7498046,2020-08-07,857913002275398,863587002,0.016932,1,98


Calculating avg price and embeddings for customer 857913002275398 and week 103
Checks for customer 857913002275398 and week 103 passed


,t_dat,customer_id,article_id,price,sales_channel_id,week_num
7580313,2020-08-15,857913002275398,636207006,0.033881,1,99
7580314,2020-08-15,857913002275398,863456003,0.025407,1,99
7498043,2020-08-07,857913002275398,720504001,0.033881,1,98
7498044,2020-08-07,857913002275398,720504008,0.033881,1,98
7498046,2020-08-07,857913002275398,863587002,0.016932,1,98


Calculating avg price and embeddings for customer 857913002275398 and week 104
Checks for customer 857913002275398 and week 104 passed
